In [1]:
# self-generated similar design, PEERS semantic matrix
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n", "") for i in items]
items = np.array(items)

In [3]:
n = 48
v = n - 8  # last 8 not tested
wordpool = np.arange(1, 1639)

nsubj = 100
g1_per = 4  # 4
g2_per = 3  # 5
g3_per = 5  # 3

### Group1: Item Recog - CR

In [4]:
df_g1_study = pd.DataFrame()
df_g1_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, g1_per * (2 * n + v), replace=False)

    for lst in range(g1_per):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()

        # study
        pres_words = all_words[lst * (2 * n + v) : lst * (2 * n + v) + 2 * n]
        pres_words = np.reshape(pres_words, (n, 2))
        pair_order = rng.permutation([0] * int(v / 2) + [1] * int(v / 2))  # 0: A-B, 1: B-A (during CR)
        tmp_df_study["study_itemno1"] = pres_words[:, 0]
        tmp_df_study["study_itemno2"] = pres_words[:, 1]
        tmp_df_study["study_item1"] = items[pres_words[:, 0] - 1]
        tmp_df_study["study_item2"] = items[pres_words[:, 1] - 1]
        tmp_df_study["order"] = np.concatenate((pair_order, [-1] * 8))
        tmp_df_study["pair_idx"] = np.arange(n) + lst * n
        tmp_df_study["list"] = lst
        tmp_df_study["session"] = subj
        tmp_df_study["subject"] = subj
        df_g1_study = pd.concat([df_g1_study, tmp_df_study])

        valid_words = pres_words[0:v, :]

        # test1: item recognition
        old_order = pair_order  # 0:B, 1:A
        old_probe = np.array([valid_words[i, 1 - old_order[i]] for i in range(v)])
        old_pair_idx = np.arange(v) + lst * n
        new_probe = all_words[lst * (2 * n + v) + 2 * n : (lst + 1) * (2 * n + v)]
        new_order = [-1] * v
        new_pair_idx = np.array([-1] * v)

        test1_probe = np.concatenate((old_probe, new_probe), axis=0)
        test1_type = np.array([1] * v + [0] * v)  # 1:old, 0:new
        test1_order = np.concatenate((old_order, new_order), axis=0)
        test1_pair_idx = np.concatenate((old_pair_idx, new_pair_idx), axis=0)
        tmp_df_test1["test_itemno1"] = test1_probe
        tmp_df_test1["test_itemno2"] = -1
        tmp_df_test1["test_item1"] = items[test1_probe - 1]
        tmp_df_test1["correct_ans"] = test1_type
        tmp_df_test1["order"] = test1_order
        tmp_df_test1["pair_idx"] = test1_pair_idx
        tmp_df_test1["test"] = 1
        tmp_df_test1["list"] = lst
        tmp_df_test1["session"] = subj
        tmp_df_test1["subject"] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

        # test2: cued recall
        cued_order = pair_order  # 0:A-B, 1:B-A
        test2_probe = np.array([valid_words[i, cued_order[i]] for i in range(v)])
        test2_ans = np.array([valid_words[i, 1 - cued_order[i]] for i in range(v)])
        tmp_df_test2["test_itemno1"] = test2_probe
        tmp_df_test2["test_itemno2"] = -1
        tmp_df_test2["test_item1"] = items[test2_probe - 1]
        tmp_df_test2["correct_ans"] = test2_ans
        tmp_df_test2["order"] = cued_order
        tmp_df_test2["pair_idx"] = np.arange(v) + lst * n
        tmp_df_test2["test"] = 2
        tmp_df_test2["list"] = lst
        tmp_df_test2["session"] = subj
        tmp_df_test2["subject"] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)
        df_g1_test = pd.concat([df_g1_test, tmp_df_test1, tmp_df_test2])

df_g1_study.reset_index(inplace=True, drop=True)
df_g1_test.reset_index(inplace=True, drop=True)
df_g1_study["group"] = 1
df_g1_test["group"] = 1

In [5]:
df_g1_study

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
0,1160,1405,RAG,SUMMIT,1,0,0,0,0,1
1,48,507,ASSISTANT,DUNGEON,0,1,0,0,0,1
2,873,945,MARS,NEST,0,2,0,0,0,1
3,835,32,LOAF,APPENDAGE,1,3,0,0,0,1
4,1125,813,PRINCESS,LETTER,0,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
19195,13,461,AIRPORT,DICE,-1,187,3,99,99,1
19196,1231,893,SANDPAPER,MILDEW,-1,188,3,99,99,1
19197,190,1495,BUILDER,TRAITOR,-1,189,3,99,99,1
19198,570,208,FIREPLACE,BUYER,-1,190,3,99,99,1


In [6]:
df_g1_test.query("session==0 and list==2")

,test_itemno1,test_itemno2,test_item1,correct_ans,order,pair_idx,test,list,session,subject,group
240,1197,-1,ROBBER,0,-1,-1,1,2,0,0,1
241,1280,-1,SHELF,1,0,126,1,2,0,0,1
242,937,-1,NAPKIN,1,0,112,1,2,0,0,1
243,1314,-1,SKELETON,0,-1,-1,1,2,0,0,1
244,139,-1,BLOUSE,1,0,131,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
355,1024,-1,PATIENT,1379,0,133,2,2,0,0,1
356,1430,-1,TAIL,536,1,129,2,2,0,0,1
357,1623,-1,WORM,270,0,105,2,2,0,0,1
358,454,-1,DETECTIVE,1455,1,100,2,2,0,0,1


### Group2: Pair Recog - CR

In [7]:
df_g2_study = pd.DataFrame()
df_g2_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, g2_per * 2 * (n + v), replace=False)

    for lst in range(g2_per):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()

        # study
        pres_words = all_words[lst * 2 * (n + v) : lst * 2 * (n + v) + 2 * n]
        pres_words = np.reshape(pres_words, (n, 2))
        pair_order = rng.permutation([0] * int(v / 2) + [1] * int(v / 2))  # 0: A-B, 1: B-A (during CR)
        tmp_df_study["study_itemno1"] = pres_words[:, 0]
        tmp_df_study["study_itemno2"] = pres_words[:, 1]
        tmp_df_study["study_item1"] = items[pres_words[:, 0] - 1]
        tmp_df_study["study_item2"] = items[pres_words[:, 1] - 1]
        tmp_df_study["order"] = np.concatenate((pair_order, [-1] * 8))
        tmp_df_study["pair_idx"] = np.arange(n) + lst * n
        tmp_df_study["list"] = lst
        tmp_df_study["session"] = subj
        tmp_df_study["subject"] = subj
        df_g2_study = pd.concat([df_g2_study, tmp_df_study])

        valid_words = pres_words[0:v, :]

        # test1: pair recognition
        old_pairs = valid_words.copy()
        old_order = pair_order  # 0:A-B, 1:B-A
        for i in range(old_pairs.shape[0]):
            if old_order[i] == 1:
                old_pairs[i] = np.flip(old_pairs[i])
        old_pair_idx = np.arange(v) + lst * n

        new_words = all_words[lst * 2 * (n + v) + 2 * n : (lst + 1) * 2 * (n + v)]
        new_pairs = np.reshape(new_words, (v, 2))
        new_order = [-1] * v
        new_pair_idx = np.array([-1] * v)

        test1_pairs = np.concatenate((old_pairs, new_pairs), axis=0)
        test1_type = np.array([1] * v + [0] * v)  # 1:old, 0:new
        test1_order = np.concatenate((old_order, new_order), axis=0)
        test1_pair_idx = np.concatenate((old_pair_idx, new_pair_idx), axis=0)
        tmp_df_test1["test_itemno1"] = test1_pairs[:, 0]
        tmp_df_test1["test_itemno2"] = test1_pairs[:, 1]
        tmp_df_test1["test_item1"] = items[test1_pairs[:, 0] - 1]
        tmp_df_test1["test_item2"] = items[test1_pairs[:, 1] - 1]
        tmp_df_test1["correct_ans"] = test1_type
        tmp_df_test1["order"] = test1_order
        tmp_df_test1["pair_idx"] = test1_pair_idx
        tmp_df_test1["test"] = 1
        tmp_df_test1["list"] = lst
        tmp_df_test1["session"] = subj
        tmp_df_test1["subject"] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

        # test2: cued recall
        cued_order = pair_order  # 0:A-B, 1:B-A
        test2_probe = np.array([valid_words[i, cued_order[i]] for i in range(v)])
        test2_ans = np.array([valid_words[i, 1 - cued_order[i]] for i in range(v)])
        tmp_df_test2["test_itemno1"] = test2_probe
        tmp_df_test2["test_itemno2"] = -1
        tmp_df_test2["test_item1"] = items[test2_probe - 1]
        tmp_df_test2["test_item2"] = -1
        tmp_df_test2["correct_ans"] = test2_ans
        tmp_df_test2["order"] = cued_order
        tmp_df_test2["pair_idx"] = np.arange(v) + lst * n
        tmp_df_test2["test"] = 2
        tmp_df_test2["list"] = lst
        tmp_df_test2["session"] = subj
        tmp_df_test2["subject"] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)
        df_g2_test = pd.concat([df_g2_test, tmp_df_test1, tmp_df_test2])

df_g2_study.reset_index(inplace=True, drop=True)
df_g2_test.reset_index(inplace=True, drop=True)
df_g2_study["group"] = 2
df_g2_test["group"] = 2

In [8]:
df_g2_study

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
0,481,1485,DOLL,TORNADO,1,0,0,0,0,2
1,472,126,DIRT,BILL,0,1,0,0,0,2
2,1509,1635,TROPHY,ZEBRA,1,2,0,0,0,2
3,1096,1013,POISON,PARENT,1,3,0,0,0,2
4,903,372,MOAT,COOK,0,4,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...
14395,848,843,LUMBER,LONDON,-1,139,2,99,99,2
14396,882,1115,MEAL,POTATO,-1,140,2,99,99,2
14397,630,1084,GENERAL,PLAQUE,-1,141,2,99,99,2
14398,617,888,GALAXY,MESSAGE,-1,142,2,99,99,2


In [9]:
df_g2_test.query("session==0 and list==2")

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,list,session,subject,group
240,1564,970,VIDEO,OCEAN,0,-1,-1,1,2,0,0,2
241,1504,1502,TRIANGLE,TREE,0,-1,-1,1,2,0,0,2
242,707,447,HIGHWAY,DENTIST,1,0,134,1,2,0,0,2
243,1480,315,TOOL,CLAM,0,-1,-1,1,2,0,0,2
244,620,1559,GANGSTER,VEST,1,0,118,1,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
355,1136,-1,PUBLICATION,-1,618,1,105,2,2,0,0,2
356,1489,-1,TOWEL,-1,968,1,100,2,2,0,0,2
357,1080,-1,PLAID,-1,1201,1,104,2,2,0,0,2
358,749,-1,JAPAN,-1,1382,0,101,2,2,0,0,2


### Group3: Association Recog - CR

In [10]:
df_g3_study = pd.DataFrame()
df_g3_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, g3_per * 2 * n, replace=False)

    for lst in range(g3_per):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()

        # study
        pres_words = all_words[lst * 2 * n : (lst + 1) * 2 * n]
        pres_words = np.reshape(pres_words, (n, 2))
        pair_order = rng.permutation([0] * int(v / 2) + [1] * int(v / 2))  # 0: A-B, 1: B-A (during CR)
        tmp_df_study["study_itemno1"] = pres_words[:, 0]
        tmp_df_study["study_itemno2"] = pres_words[:, 1]
        tmp_df_study["study_item1"] = items[pres_words[:, 0] - 1]
        tmp_df_study["study_item2"] = items[pres_words[:, 1] - 1]
        tmp_df_study["order"] = np.concatenate((pair_order, [-1] * 8))
        tmp_df_study["pair_idx"] = np.arange(n) + lst * n
        tmp_df_study["list"] = lst
        tmp_df_study["session"] = subj
        tmp_df_study["subject"] = subj
        df_g3_study = pd.concat([df_g3_study, tmp_df_study])

        valid_words = pres_words[0:v, :]

        # test1: association recognition
        intact_idx = rng.permutation([True] * int(v / 2) + [False] * int(v / 2))
        intact_pair_idx = np.arange(v)[intact_idx] + lst * n
        intact_pairs_raw = valid_words[intact_idx, :]
        intact_pairs = valid_words[intact_idx, :]
        intact_order = pair_order[intact_idx]  # 0:A-B, 1:B-A
        for i in range(intact_pairs.shape[0]):
            if intact_order[i] == 1:
                intact_pairs[i] = np.flip(intact_pairs[i])

        rearrange_idx = ~intact_idx
        rearrange_pair_idx = np.array([-1] * int(v / 2))
        rearrange_pairs = valid_words[rearrange_idx, :]
        rearrange_pairs[:, 1] = rng.permutation(rearrange_pairs[:, 1])
        rearrange_order = pair_order[rearrange_idx]  # 0:A-B, 1:B-A
        for i in range(rearrange_pairs.shape[0]):
            if rearrange_order[i] == 1:
                rearrange_pairs[i] = np.flip(rearrange_pairs[i])

        test1_pairs = np.concatenate((intact_pairs, rearrange_pairs), axis=0)
        test1_pair_idx = np.concatenate((intact_pair_idx, rearrange_pair_idx), axis=0)
        test1_type = np.array([1] * int(v / 2) + [0] * int(v / 2))  # 1:intact, 0:rearranged
        test1_order = np.concatenate((intact_order, rearrange_order), axis=0)
        tmp_df_test1["test_itemno1"] = test1_pairs[:, 0]
        tmp_df_test1["test_itemno2"] = test1_pairs[:, 1]
        tmp_df_test1["test_item1"] = items[test1_pairs[:, 0] - 1]
        tmp_df_test1["test_item2"] = items[test1_pairs[:, 1] - 1]
        tmp_df_test1["correct_ans"] = test1_type
        tmp_df_test1["order"] = test1_order
        tmp_df_test1["pair_idx"] = test1_pair_idx
        tmp_df_test1["test"] = 1
        tmp_df_test1["list"] = lst
        tmp_df_test1["session"] = subj
        tmp_df_test1["subject"] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

        # test2: cued recall
        cued_order = pair_order[intact_idx]  # 0:A-B, 1:B-A
        test2_probe = np.array([intact_pairs_raw[i, cued_order[i]] for i in range(int(v / 2))])
        test2_ans = np.array([intact_pairs_raw[i, 1 - cued_order[i]] for i in range(int(v / 2))])
        tmp_df_test2["test_itemno1"] = test2_probe
        tmp_df_test2["test_itemno2"] = -1
        tmp_df_test2["test_item1"] = items[test2_probe - 1]
        tmp_df_test2["test_item2"] = -1
        tmp_df_test2["correct_ans"] = test2_ans
        tmp_df_test2["order"] = cued_order
        tmp_df_test2["pair_idx"] = intact_pair_idx
        tmp_df_test2["test"] = 2
        tmp_df_test2["list"] = lst
        tmp_df_test2["session"] = subj
        tmp_df_test2["subject"] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)
        df_g3_test = pd.concat([df_g3_test, tmp_df_test1, tmp_df_test2])

df_g3_study.reset_index(inplace=True, drop=True)
df_g3_test.reset_index(inplace=True, drop=True)
df_g3_study["group"] = 3
df_g3_test["group"] = 3

In [11]:
df_g3_study.query("session==1")

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
240,354,1026,COMMANDER,PATRIOT,1,0,0,1,1,3
241,947,934,NEUTRON,MUSHROOM,1,1,0,1,1,3
242,1072,546,PIN,EYELASH,0,2,0,1,1,3
243,1376,735,STEM,INCENSE,0,3,0,1,1,3
244,867,1334,MARBLE,SNEAKER,1,4,0,1,1,3
...,...,...,...,...,...,...,...,...,...,...
475,536,818,ENGINEER,LIEUTENANT,-1,235,4,1,1,3
476,77,373,BANANA,COOKBOOK,-1,236,4,1,1,3
477,845,329,LOUNGE,CLOVE,-1,237,4,1,1,3
478,783,273,KNAPSACK,CHAIN,-1,238,4,1,1,3


In [12]:
df_g3_test.query("session==0 and list==4")

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,list,session,subject,group
240,538,1276,ERASER,SHEARS,1,0,222,1,4,0,0,3
241,358,1421,COMPASS,SWING,1,1,220,1,4,0,0,3
242,1638,883,ZUCCHINI,MEAT,0,1,-1,1,4,0,0,3
243,32,441,APPENDAGE,DASHBOARD,1,0,229,1,4,0,0,3
244,970,799,OCEAN,LANDSCAPE,1,0,201,1,4,0,0,3
245,826,1314,LINT,SKELETON,1,1,204,1,4,0,0,3
246,1110,1489,PORT,TOWEL,1,1,213,1,4,0,0,3
247,1172,1527,REBEL,TWIG,1,1,198,1,4,0,0,3
248,570,1387,FIREPLACE,STORE,0,0,-1,1,4,0,0,3
249,988,514,OUTFIT,EARRING,0,0,-1,1,4,0,0,3


In [13]:
df_study = pd.concat([df_g1_study, df_g2_study, df_g3_study])
df_test = pd.concat([df_g1_test, df_g2_test, df_g3_test])

In [14]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
0,1160,1405,RAG,SUMMIT,1,0,0,0,0,1
1,48,507,ASSISTANT,DUNGEON,0,1,0,0,0,1
2,873,945,MARS,NEST,0,2,0,0,0,1
3,835,32,LOAF,APPENDAGE,1,3,0,0,0,1
4,1125,813,PRINCESS,LETTER,0,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
23995,838,846,LOCK,LOVER,-1,235,4,99,99,3
23996,415,503,CROWN,DRYER,-1,236,4,99,99,3
23997,694,404,HAYSTACK,CREEK,-1,237,4,99,99,3
23998,810,112,LEMON,BEE,-1,238,4,99,99,3


In [15]:
df_test

,test_itemno1,test_itemno2,test_item1,correct_ans,order,pair_idx,test,list,session,subject,group,test_item2
0,609,-1,FROST,1,0,14,1,0,0,0,1,NaN
1,101,-1,BATTERY,0,-1,-1,1,0,0,0,1,NaN
2,927,-1,MOVIE,1,1,13,1,0,0,0,1,NaN
3,1303,-1,SIDEWALK,1,0,31,1,0,0,0,1,NaN
4,945,-1,NEST,1,0,2,1,0,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1169,-1,RAT,331,0,226,2,4,99,99,3,-1
29996,1349,-1,SPHINX,848,1,219,2,4,99,99,3,-1
29997,1307,-1,SILK,72,0,194,2,4,99,99,3,-1
29998,984,-1,ORGAN,899,1,196,2,4,99,99,3,-1


In [16]:
# save
with open("simuS1_data/simuS1_design.pkl", "wb") as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)